# Dash

# Hubble Diagram + SN Light Curves + Fits

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import matplotlib.colors as Colors

# Colors
def make_color_tuple( color ):
    """
    turn something like "#000000" into 0,0,0
    or "#FFFFFF into "255,255,255"
    """
    R = color[1:3]
    G = color[3:5]
    B = color[5:7]

    R = int(R, 16)
    G = int(G, 16)
    B = int(B, 16)

    return R,G,B

band_list = ['DESg', 'DESr', 'DESi', 'DESz', 'VISTAY', 'VISTAJ', 'VISTAH', 'VISTAK']
colors_str = ["red", "olive", "darkred", "goldenrod", "skyblue", "orange", "salmon", 'magenta']
colors = [make_color_tuple(Colors.CSS4_COLORS[key]) for key in colors_str]

app = dash.Dash()

# HD data
df = pd.read_csv('sample.csv')

app.layout = html.Div(children=[
    html.Div(
        dcc.Graph(
            id='HD',
            figure={
                'data': [go.Scatter(                 
                    x=df[df['rest_band'] == band]['z'],
                    y=df[df['rest_band'] == band]['mmax'],
                    text=df[df['rest_band'] == band]['SN'],
                    mode='markers',
                    opacity=0.7,
                    marker={
                        'size': 15,
                        'color': 'rgb%s'%str(colors[band_list.index(band)]),
                        'line': {'width': 0.5, 'color': 'white'}
                            },
                    name=band
                                    ) for i, band in enumerate(df.rest_band.unique())
                        ],
                'layout': go.Layout(
                    title='Hubble Diagram',
                    xaxis={'type': 'linear', 'title': 'z'},
                    yaxis={'title': 'mag'},
                    legend={'x': 0, 'y': 1},
                    margin={'l': 40, 'b': 40, 't': 30, 'r': 10},
                    hovermode='closest'
                                    )
                    }
                )
            ),
    
    html.Div(children='''SN name:'''),
    
    dcc.Input(id='input', value='', type='text'),
    
    html.Div(id='output-graph'),
])


@app.callback(
    Output(component_id='output-graph', component_property='children'),
    [Input(component_id='input', component_property='value')]
            )


def update_value(input_data):
    # SN data
    df1 = pd.read_csv('data/' + input_data + '_flux.dat')
    df1.reset_index(inplace=True)
    
    data1 = [go.Scatter(
                x=df1[df1['band'] == band]['MJD'],
                y=df1[df1['band'] == band]['flux'],
                error_y=dict(
                    type='data',
                    array=df1[df1['band'] == band]['flux_err'],
                    visible=True,
                    color = 'rgb%s'%str(colors[band_list.index(band)])),
                text=df1[df1['band'] == band]['band'],
                mode='markers',
                opacity=0.7,
                marker={
                    'size': 15,
                    'color': 'rgb%s'%str(colors[band_list.index(band)]),
                    'line': {'width': 0.5, 'color': 'white'}
                        },
                name=band,
                        ) for i, band in enumerate(df1.band.unique())
            ]
    
    # SN fits
    df2 = pd.read_csv('fits/' + input_data + '_fits.dat')
    df2.reset_index(inplace=True)
    
    data2 = [go.Scatter(
                x=df2[df2['band'] == band]['MJD'],
                y=df2[df2['band'] == band]['flux'],
                text=df2[df2['band'] == band]['band'],
                mode='lines',
                marker={'color': 'rgb%s'%str(colors[band_list.index(band)])},
                name=band,
                        ) for i, band in enumerate(df2.band.unique())
            ]
     
    data = data1 + data2

    return html.Div(
            dcc.Graph(
                id='SNlc',
                figure={
                    'data': data,
                    'layout': go.Layout(
                        title=input_data,
                        xaxis={'title': 'MJD'},
                        yaxis={'title': 'flux'},
                        legend={'x': 0, 'y': 1},
                        margin={'l': 40, 'b': 40, 't': 40, 'r': 10},
                        hovermode='closest'
                                        )
                        }
                    )
           
                    )
    

if __name__ == '__main__':
    app.run_server(debug=False)